# ACTIVIDAD SESIÓN 11

**Utiliza el dataset del proyecto final o si prefieres cualquier otro para hacer un ejemplo de red neuronal mediante Tensorflow, Keras u otro framework visto en el curso**

Se utilizará el dataset del Titanic

## Obtención de datos

Hemos usado los dos dataset que ya venían separados en train y test.

In [1]:
## Data Preparation and Modeling
import pandas as pd
import numpy as np
# Fijamos la semilla para optener siempre los mismos datos
np.random.seed(0)
train = pd.read_csv('../Sesion7/titanic_train.csv')
test = pd.read_csv('../Sesion7/titanic_test.csv', names=train.columns.values)
train.tail(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
688,689,0,3,"Fischer, Mr. Eberhard Thelander",male,18.0,0,0,350036,7.7958,NaN,S
689,690,1,1,"Madill, Miss. Georgette Alexandra",female,15.0,0,1,24160,211.3375,B5,S
690,691,1,1,"Dick, Mr. Albert Adrian",male,31.0,1,0,17474,57.0000,B20,S
691,692,1,3,"Karun, Miss. Manca",female,4.0,0,1,349256,13.4167,NaN,C
692,693,1,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,NaN,S
693,694,0,3,"Saad, Mr. Khalil",male,25.0,0,0,2672,7.2250,NaN,C
694,695,0,1,"Weir, Col. John",male,60.0,0,0,113800,26.5500,NaN,S
695,696,0,2,"Chapman, Mr. Charles Henry",male,52.0,0,0,248731,13.5000,NaN,S
696,697,0,3,"Kelly, Mr. James",male,44.0,0,0,363592,8.0500,NaN,S
697,698,1,3,"Mullens, Miss. Katherine ""Katie""",female,NaN,0,0,35852,7.7333,NaN,Q


Nos quedamos con aquellas variables que consideramos importantes para nuestro problema: Eliminamos Name (el nombre no es decisivo sobre si sobrevives o no), PassengerId (el identificador de pasajero no es decisivo sobre si sobrevives o no), Ticket (puede informar sobre la zona donde tiene el camarote, pero es complicada de extraer, hay que realizar mucho procesamiento y consideramos mejor eliminarla, pero es un buen punto de mejora) y Cabin (a simple vista tiene muchos valores perdidos).

In [2]:
# Nos quedamos con aquellas variables que consideramos importantes para nuestro problema:
# Eliminamos Name, PassengerId, Ticket y Cabin
train = train.loc[:, ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test = test.loc[:, ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [3]:
## Separamos en conjunto de test y train
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']

X_test = test.drop('Survived', axis=1)
y_test = test['Survived']

## PROCESAMIENTO DE DATOS

Convertimos las variables categoricas en dummies.

In [4]:
numeric_features = train.select_dtypes(include=['int64', 'float64']).drop(['Survived'], axis=1).columns
numeric_features

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [5]:
categorical_features = train.select_dtypes(include=['object']).columns
categorical_features

Index(['Sex', 'Embarked'], dtype='object')

In [6]:
# Convertimos las variables categoricas en dummies
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_train

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
693,3,25.0,0,0,7.2250,0,1,1,0,0
694,1,60.0,0,0,26.5500,0,1,0,0,1
695,2,52.0,0,0,13.5000,0,1,0,0,1
696,3,44.0,0,0,8.0500,0,1,0,0,1


Estudiamos los valores perdidos

In [7]:
def ver_null(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending = False)
    missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Porcentaje'])
    return missing_data
ver_null(X_train)

,Total,Porcentaje
Age,145,0.207736
Embarked_S,0,0.000000
Embarked_Q,0,0.000000
Embarked_C,0,0.000000
Sex_male,0,0.000000
Sex_female,0,0.000000
Fare,0,0.000000
Parch,0,0.000000
SibSp,0,0.000000
Pclass,0,0.000000


In [8]:
# Eliminamos la columna Age ya que tiene muchos NaN
X_train = X_train.drop(['Age'], axis=1)
X_test = X_test.drop(['Age'], axis=1)

In [9]:
# Convertimos a array
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')

## Aplicamos un modelo de clasificación

Definimos el modelo, entrenamos y obtenemos resultados. En nuestro caso creamos una red neuronal de 4 capas con activación sigmoide en la capa de salida y activación relu en las capas intermedias. Obviamente necesitamos dimension 9 de entrada de la primera capa, ya que disponemos de 9 variables o columnas del dataset una vez preprocesado.

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.losses import MeanSquaredError

model = Sequential()
model.add(Dense(16, input_dim=9, activation='relu'))
model.add(Dense(16, input_dim=5, activation='relu'))
model.add(Dense(16, input_dim=10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=MeanSquaredError(),
              optimizer='adam',
              metrics=['binary_accuracy'])

Using TensorFlow backend.
/home/alberto/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alberto/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alberto/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alberto/.local/lib/python3.6/site-packages/tensorflow/py

Entrenamos el modelo

In [11]:
model.fit(np.array(X_train), y_train, epochs=1000)


Epoch 1/1000
698/698 [==============================] - 0s 251us/step - loss: 0.2641 - binary_accuracy: 0.6562
Epoch 2/1000
698/698 [==============================] - 0s 45us/step - loss: 0.2212 - binary_accuracy: 0.6848
Epoch 3/1000
698/698 [==============================] - 0s 45us/step - loss: 0.2072 - binary_accuracy: 0.6805
Epoch 4/1000
698/698 [==============================] - 0s 37us/step - loss: 0.1967 - binary_accuracy: 0.7006
Epoch 5/1000
698/698 [==============================] - 0s 41us/step - loss: 0.1928 - binary_accuracy: 0.6991
Epoch 6/1000
698/698 [==============================] - 0s 37us/step - loss: 0.1880 - binary_accuracy: 0.7077
Epoch 7/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1844 - binary_accuracy: 0.7063
Epoch 8/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1824 - binary_accuracy: 0.7393
Epoch 9/1000
698/698 [==============================] - 0s 44us/step - loss: 0.1750 - binary_accuracy: 0.7464
Epoch 10

Epoch 73/1000
698/698 [==============================] - 0s 36us/step - loss: 0.1508 - binary_accuracy: 0.7865
Epoch 74/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1459 - binary_accuracy: 0.7951
Epoch 75/1000
698/698 [==============================] - 0s 37us/step - loss: 0.1468 - binary_accuracy: 0.7980
Epoch 76/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1457 - binary_accuracy: 0.7980
Epoch 77/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1490 - binary_accuracy: 0.7994
Epoch 78/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1575 - binary_accuracy: 0.7794
Epoch 79/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1520 - binary_accuracy: 0.8052
Epoch 80/1000
698/698 [==============================] - 0s 37us/step - loss: 0.1462 - binary_accuracy: 0.7994
Epoch 81/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1459 - binary_accuracy: 0.8009
E

698/698 [==============================] - 0s 38us/step - loss: 0.1413 - binary_accuracy: 0.8080
Epoch 146/1000
698/698 [==============================] - 0s 44us/step - loss: 0.1424 - binary_accuracy: 0.8009
Epoch 147/1000
698/698 [==============================] - 0s 40us/step - loss: 0.1380 - binary_accuracy: 0.8152
Epoch 148/1000
698/698 [==============================] - 0s 38us/step - loss: 0.1428 - binary_accuracy: 0.8123
Epoch 149/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1387 - binary_accuracy: 0.8152
Epoch 150/1000
698/698 [==============================] - 0s 41us/step - loss: 0.1404 - binary_accuracy: 0.8095
Epoch 151/1000
698/698 [==============================] - 0s 44us/step - loss: 0.1443 - binary_accuracy: 0.7966
Epoch 152/1000
698/698 [==============================] - 0s 31us/step - loss: 0.1464 - binary_accuracy: 0.7994
Epoch 153/1000
698/698 [==============================] - 0s 31us/step - loss: 0.1417 - binary_accuracy: 0.8080
Epoch 1

698/698 [==============================] - 0s 34us/step - loss: 0.1336 - binary_accuracy: 0.8138
Epoch 219/1000
698/698 [==============================] - 0s 30us/step - loss: 0.1373 - binary_accuracy: 0.8095
Epoch 220/1000
698/698 [==============================] - 0s 32us/step - loss: 0.1343 - binary_accuracy: 0.8109
Epoch 221/1000
698/698 [==============================] - 0s 36us/step - loss: 0.1342 - binary_accuracy: 0.8152
Epoch 222/1000
698/698 [==============================] - 0s 32us/step - loss: 0.1364 - binary_accuracy: 0.8095
Epoch 223/1000
698/698 [==============================] - 0s 32us/step - loss: 0.1321 - binary_accuracy: 0.8123
Epoch 224/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1326 - binary_accuracy: 0.8195
Epoch 225/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1322 - binary_accuracy: 0.8195
Epoch 226/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1348 - binary_accuracy: 0.8123
Epoch 2

698/698 [==============================] - 0s 34us/step - loss: 0.1299 - binary_accuracy: 0.8252
Epoch 292/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1333 - binary_accuracy: 0.8152
Epoch 293/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1322 - binary_accuracy: 0.8152
Epoch 294/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1290 - binary_accuracy: 0.8238
Epoch 295/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1323 - binary_accuracy: 0.8181
Epoch 296/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1314 - binary_accuracy: 0.8152
Epoch 297/1000
698/698 [==============================] - 0s 37us/step - loss: 0.1309 - binary_accuracy: 0.8181
Epoch 298/1000
698/698 [==============================] - 0s 38us/step - loss: 0.1313 - binary_accuracy: 0.8152
Epoch 299/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1331 - binary_accuracy: 0.8195
Epoch 3

...

698/698 [==============================] - 0s 35us/step - loss: 0.1223 - binary_accuracy: 0.8238
Epoch 657/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1229 - binary_accuracy: 0.8309
Epoch 658/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1232 - binary_accuracy: 0.8324
Epoch 659/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1231 - binary_accuracy: 0.8238
Epoch 660/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1223 - binary_accuracy: 0.8309
Epoch 661/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1214 - binary_accuracy: 0.8295
Epoch 662/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1206 - binary_accuracy: 0.8309
Epoch 663/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1272 - binary_accuracy: 0.8367
Epoch 664/1000
698/698 [==============================] - 0s 36us/step - loss: 0.1228 - binary_accuracy: 0.8324
Epoch 6

698/698 [==============================] - 0s 37us/step - loss: 0.1216 - binary_accuracy: 0.8295
Epoch 730/1000
698/698 [==============================] - 0s 36us/step - loss: 0.1213 - binary_accuracy: 0.8352
Epoch 731/1000
698/698 [==============================] - 0s 42us/step - loss: 0.1208 - binary_accuracy: 0.8281
Epoch 732/1000
698/698 [==============================] - 0s 36us/step - loss: 0.1214 - binary_accuracy: 0.8338
Epoch 733/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1219 - binary_accuracy: 0.8324
Epoch 734/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1225 - binary_accuracy: 0.8324
Epoch 735/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1228 - binary_accuracy: 0.8266
Epoch 736/1000
698/698 [==============================] - 0s 53us/step - loss: 0.1227 - binary_accuracy: 0.8324
Epoch 737/1000
698/698 [==============================] - 0s 65us/step - loss: 0.1230 - binary_accuracy: 0.8367
Epoch 7

698/698 [==============================] - 0s 35us/step - loss: 0.1179 - binary_accuracy: 0.8352
Epoch 803/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1188 - binary_accuracy: 0.8295
Epoch 804/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1204 - binary_accuracy: 0.8266
Epoch 805/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1217 - binary_accuracy: 0.8381
Epoch 806/1000
698/698 [==============================] - 0s 38us/step - loss: 0.1193 - binary_accuracy: 0.8367
Epoch 807/1000
698/698 [==============================] - 0s 36us/step - loss: 0.1221 - binary_accuracy: 0.8238
Epoch 808/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1244 - binary_accuracy: 0.8309
Epoch 809/1000
698/698 [==============================] - 0s 42us/step - loss: 0.1206 - binary_accuracy: 0.8381
Epoch 810/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1208 - binary_accuracy: 0.8352
Epoch 8

698/698 [==============================] - 0s 32us/step - loss: 0.1174 - binary_accuracy: 0.8424
Epoch 876/1000
698/698 [==============================] - 0s 39us/step - loss: 0.1191 - binary_accuracy: 0.8338
Epoch 877/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1243 - binary_accuracy: 0.8295
Epoch 878/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1206 - binary_accuracy: 0.8395
Epoch 879/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1225 - binary_accuracy: 0.8266
Epoch 880/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1184 - binary_accuracy: 0.8395
Epoch 881/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1191 - binary_accuracy: 0.8410
Epoch 882/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1181 - binary_accuracy: 0.8381
Epoch 883/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1177 - binary_accuracy: 0.8410
Epoch 8

698/698 [==============================] - 0s 33us/step - loss: 0.1179 - binary_accuracy: 0.8381
Epoch 949/1000
698/698 [==============================] - 0s 32us/step - loss: 0.1194 - binary_accuracy: 0.8453
Epoch 950/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1164 - binary_accuracy: 0.8424
Epoch 951/1000
698/698 [==============================] - 0s 36us/step - loss: 0.1212 - binary_accuracy: 0.8367
Epoch 952/1000
698/698 [==============================] - 0s 33us/step - loss: 0.1186 - binary_accuracy: 0.8367
Epoch 953/1000
698/698 [==============================] - 0s 38us/step - loss: 0.1212 - binary_accuracy: 0.8395
Epoch 954/1000
698/698 [==============================] - 0s 35us/step - loss: 0.1185 - binary_accuracy: 0.8338
Epoch 955/1000
698/698 [==============================] - 0s 34us/step - loss: 0.1175 - binary_accuracy: 0.8424
Epoch 956/1000
698/698 [==============================] - 0s 41us/step - loss: 0.1168 - binary_accuracy: 0.8381
Epoch 9

In [12]:
# Evaluamos el modelo
scores = model.evaluate(X_train, y_train)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

698/698 [==============================] - 0s 46us/step

binary_accuracy: 84.67%


Mostramos el resultado de la prediccion con los datos de test

In [13]:
print (model.predict(X_test).round())

[[0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.

Vemos que hemos tenido un resultado notable (mas del 80% de datos predecidos). Es análogo al que obtuvimos con RandomForest en la unidad 7. Así podemos variar la estructura de la red para lograr mejores resultados y que se adapte mejor a nuestro problema como posible mejora o preprocesar mucho mejor los datos, sin embargo, para el objetivo del ejercicio hemos completado lo que queríamos.